In [1]:
import sympy
from sympy import init_printing
from sympy import MutableDenseMatrix
# init_printing(use_latex='png')

# Use IPython's Latex capabilities to avoid a bug in Sympy's matrix printing
from IPython.display import display, Math, Latex

import numpy as np
import pyamg
from scipy import sparse
from scipy import ndimage
import rasterio as rio
from geoblend.coefficient_matrix import create_coefficient_matrix

import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rcParams['image.interpolation'] = 'none'
mpl.rcParams['image.cmap'] = 'cubehelix'

%matplotlib inline

np.set_printoptions(linewidth=200)

In [2]:
def display_matrix(m):
    """
    Helper function to display the coefficient
    matrix.
    """
    
    if type(m) == MutableDenseMatrix:
        tex = sympy.latex(m)
    elif type(m) == np.ndarray:
        tex = sympy.latex(sympy.Matrix(mask.astype(np.int8).tolist()))
    else:
        # Crazy-ness to get integer represention ...
        tex = sympy.latex(sympy.Matrix(m.toarray().astype(np.int8).tolist()))
    
    display(Math(tex))

In [3]:
def create_auxiliary_equation(mask):
    """
    Create a sympy representation of the auxiliary equation corresponding
    to the Poisson equation.
    
    This prevents me from tedious, error-proned, hand-written algebra.
    """
    
    height, width = mask.shape
    x_domain = range(width)
    y_domain = range(height)
    
    # Create symbolic variables representing the unknown image
    # and the known gradient that will be imposed on the unknown image.
    
    # Note: One variable per pixel. f and g are pixel-aligned.
    f = [ sympy.symbols(' '.join([ 'f%d%d' % (i, j) for j in y_domain ])) for i in x_domain ]
    g = [ sympy.symbols(' '.join([ 'g%d%d' % (i, j) for j in y_domain ])) for i in x_domain ]
    c = [ sympy.symbols(' '.join([ 'c%d%d' % (i, j) for j in y_domain ])) for i in x_domain ]
    
    # Create the variable that will comprise the auxiliary equation.
    equation = 0
    
    for j in y_domain:
        for i in x_domain:
            
            if mask[j][i] == 0:
                continue
            
            #
            # Define the index of the 4-connected neighbors
            #
            
            # For a pixel p with indices (i, j), the
            # 4-connected neighbors are defined by the
            # surrounding 3x3 pixel grid as:
            #
            # - N - 
            # W p E
            # - S - 
            
            nj, ni = j - 1, i
            sj, si = j + 1, i
            ej, ei = j, i + 1
            wj, wi = j, i - 1
            
            # TODO: This pays no attention to boundaries. The gradient
            #       at the boundary should never be sampled.
            if nj in y_domain:
                equation += (f[i][j] - f[ni][nj] - (g[i][j] - g[ni][nj])) ** 2
            if ei in x_domain:
                equation += (f[i][j] - f[ei][ej] - (g[i][j] - g[ei][ej])) ** 2
            if sj in y_domain:
                equation += (f[i][j] - f[si][sj] - (g[i][j] - g[si][sj])) ** 2
            if wi in x_domain:
                equation += (f[i][j] - f[wi][wj] - (g[i][j] - g[wi][wj])) ** 2
    
    return equation, f, g, c

In [4]:
mask1 = np.array([
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 1, 1, 1, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 1, 1, 1, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
], dtype=np.uint8)

In [5]:
mask2 = np.array([
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
  [0, 0, 1, 1, 1, 0, 0, 0, 0, 0],
  [0, 0, 1, 1, 1, 1, 1, 1, 0, 0],
  [0, 0, 0, 1, 1, 1, 1, 1, 1, 0],
  [0, 0, 0, 1, 1, 1, 1, 1, 0, 0],
  [0, 0, 0, 0, 1, 1, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
], dtype=np.uint8)
mask2

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 1, 1, 1, 1, 0, 0],
       [0, 0, 0, 1, 1, 1, 1, 1, 1, 0],
       [0, 0, 0, 1, 1, 1, 1, 1, 0, 0],
       [0, 0, 0, 0, 1, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=uint8)

In [6]:
with rio.drivers():
    with rio.open("test2/20150705_014439_081f_analytic.tif") as src:
        mask3 = src.read(4).astype(np.uint8)
        mask3[np.nonzero(mask3)] = 1
mask3

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
       [0,

In [7]:
eq1, f1, g1, c1 = create_auxiliary_equation(mask1)
eq2, f2, g2, c2 = create_auxiliary_equation(mask2)
eq3, f3, g3, c3 = create_auxiliary_equation(mask3)

In [8]:
def linearize_system(mask, equation, f):
    
    indices = np.nonzero(mask)
    return [
        sympy.diff(equation, f[idx[1]][idx[0]])
        for idx in zip(*indices)
    ]

In [9]:
system1 = linearize_system(mask1, eq1, f1)
system2 = linearize_system(mask2, eq2, f2)
system3 = linearize_system(mask3, eq3, f3)

In [10]:
mat1 = sympy.Matrix(system1)
mat2 = sympy.Matrix(system2)
mat3 = sympy.Matrix(system3)

In [11]:
# display_matrix(mat1)
# display_matrix(mat2)
# display_matrix(mat3)

In [12]:
def get_boundary_mask(m):
    selem = np.array([
        [0, 1, 0],
        [1, -4, 1],
        [0, 1, 0]
    ])
    b = ndimage.convolve(m.astype(np.float), selem, mode='constant', cval=0)
    b[b < 0] = 0
    return b

In [13]:
get_boundary_mask(mask1)

array([[ 0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.]])

In [14]:
get_boundary_mask(mask2)

array([[ 0.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  2.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  2.,  1.,  1.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  2.,  0.],
       [ 0.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  2.,  0.],
       [ 0.,  0.,  0.,  2.,  0.,  0.,  2.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  0.]])

In [18]:
get_boundary_mask(mask3).astype(np.int8)

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0],
       [0,

In [19]:
def get_symbolic_replacements(mask, f, g, c):
    
    # Replace boundaries with symbolic values
    boundary = get_boundary_mask(mask)
    bindices = np.nonzero(boundary)
    
    return [
        (f[idx[1]][idx[0]], c[idx[1]][idx[0]])
        for idx in zip(*bindices)
    ]

In [20]:
symbolic_replacements1 = get_symbolic_replacements(mask1, f1, g1, c1)
symbolic_replacements2 = get_symbolic_replacements(mask2, f2, g2, c2)
symbolic_replacements3 = get_symbolic_replacements(mask3, f3, g3, c3)

In [21]:
print mask3
# display_matrix(mat1.subs(symbolic_replacements1))
# display_matrix(mat2.subs(symbolic_replacements2))
display_matrix(mat3.subs(symbolic_replacements3))

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0]
 [0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0]
 [0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0]
 [0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0]
 [0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0]
 [0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0

<IPython.core.display.Math object>

In [22]:
def get_unknown_vector(mask, f):
    
    indices = np.nonzero(mask)
    return [
        f[idx[1]][idx[0]]
        for idx in zip(*indices)
    ]

In [23]:
vec1 = get_unknown_vector(mask1, f1)
vec2 = get_unknown_vector(mask2, f2)
vec3 = get_unknown_vector(mask3, f3)

In [24]:
coeff1 = mat1.jacobian(vec1)
coeff2 = mat2.jacobian(vec2)
coeff3 = mat3.jacobian(vec3)

In [25]:
# display_matrix(coeff1)
# display_matrix(coeff2)
# display_matrix(coeff3)
coeff3

Matrix([
[12, -4,  0,  0,  0,  0,  0,  0,  0,  0, -4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0

In [26]:
a = np.array(coeff3.tolist())
np.save('matrix.npy', a)

In [95]:
# mask3

### Sample data

In [27]:
def get_replacements(mask, f, g, reference, field):
    
    # Replace boundaries with values from the reference image
    boundary = get_boundary_mask(mask)
    bindices = np.nonzero(boundary)
    
    boundary_replacements = [
        (f[idx[1]][idx[0]], reference[idx])
        for idx in zip(*bindices)
    ]
    
    # Replace all gradient values
    m = np.ones_like(mask)
    indices = np.nonzero(m)
    
    field_replacements = [
        (g[idx[1]][idx[0]], field[idx])
        for idx in zip(*indices)
    ]
    
    return boundary_replacements + field_replacements

#### Rectangular case

In [27]:
source1 = np.array([
    [502, 527, 545, 517, 518, 492, 457, 562, 405, 420],
    [605, 512, 444, 473, 465, 496, 527, 445, 387, 397],
    [543, 446, 440, 393, 491, 472, 471, 417, 439, 371],
    [513, 476, 494, 448, 470, 491, 492, 443, 559, 514],
    [454, 487, 498, 471, 402, 484, 471, 377, 574, 452],
    [507, 478, 499, 484, 381, 372, 249, 333, 607, 410],
    [451, 497, 497, 392, 389, 476, 357, 366, 400, 464],
    [485, 517, 567, 531, 443, 324, 370, 408, 361, 464]
], dtype=np.uint16)

reference1 = np.array([
    [611, 573, 639, 564, 626, 588, 556, 503, 458, 461],
    [689, 559, 532, 550, 572, 601, 521, 466, 469, 437],
    [631, 530, 513, 504, 545, 516, 428, 444, 447, 430],
    [648, 566, 518, 514, 592, 537, 518, 468, 658, 559],
    [553, 587, 556, 544, 423, 574, 546, 452, 456, 387],
    [590, 598, 583, 564, 408, 389, 219, 498, 501, 479],
    [565, 572, 564, 436, 442, 638, 208, 382, 466, 455],
    [566, 545, 570, 507, 429, 378, 425, 474, 425, 466]
], dtype=np.uint16)

field1 = np.array([
    [-130.,   70.,  154.,  121.,   95., -102., -127.,  349., -268.,  107.],
    [ 177.,   26., -194.,   73., -118.,   28.,  239., -113., -138.,  -44.],
    [  57., -187.,  -17., -280.,  164.,  -61.,  -24., -130.,   22., -273.],
    [ 100.,  -36.,  114.,  -36.,   48.,   46.,   92.,  -73.,  266.,  314.],
    [-215.,   42.,   41.,   52., -198.,  200.,  282., -313.,  301., -150.],
    [ 111.,  -78.,   39.,  193., -123., -102., -537., -267.,  711., -422.],
    [-187.,   45.,   33., -333., -136.,  462.,  -33.,  -34., -198.,  202.],
    [-138.,   40.,  253.,  282.,  109., -419.,  -45.,  176., -169.,  248.]
], dtype=np.float)

In [111]:
replacements1 = get_replacements(mask1, f1, g1, reference1, field1)

In [113]:
# display_matrix(mat1.subs(replacements1))
display_matrix(mat1.subs(replacements1))

<IPython.core.display.Math object>

#### Non-rectangular small case

In [28]:
# Data is the same as rectangular case. Mask is different above.

source2 = np.array([
    [502, 527, 545, 517, 518, 492, 457, 562, 405, 420],
    [605, 512, 444, 473, 465, 496, 527, 445, 387, 397],
    [543, 446, 440, 393, 491, 472, 471, 417, 439, 371],
    [513, 476, 494, 448, 470, 491, 492, 443, 559, 514],
    [454, 487, 498, 471, 402, 484, 471, 377, 574, 452],
    [507, 478, 499, 484, 381, 372, 249, 333, 607, 410],
    [451, 497, 497, 392, 389, 476, 357, 366, 400, 464],
    [485, 517, 567, 531, 443, 324, 370, 408, 361, 464]
], dtype=np.uint16)

reference2 = np.array([
    [611, 573, 639, 564, 626, 588, 556, 503, 458, 461],
    [689, 559, 532, 550, 572, 601, 521, 466, 469, 437],
    [631, 530, 513, 504, 545, 516, 428, 444, 447, 430],
    [648, 566, 518, 514, 592, 537, 518, 468, 658, 559],
    [553, 587, 556, 544, 423, 574, 546, 452, 456, 387],
    [590, 598, 583, 564, 408, 389, 219, 498, 501, 479],
    [565, 572, 564, 436, 442, 638, 208, 382, 466, 455],
    [566, 545, 570, 507, 429, 378, 425, 474, 425, 466]
], dtype=np.uint16)

field2 = np.array([
    [-130.,   70.,  154.,  121.,   95., -102., -127.,  349., -268.,  107.],
    [ 177.,   26., -194.,   73., -118.,   28.,  239., -113., -138.,  -44.],
    [  57., -187.,  -17., -280.,  164.,  -61.,  -24., -130.,   22., -273.],
    [ 100.,  -36.,  114.,  -36.,   48.,   46.,   92.,  -73.,  266.,  314.],
    [-215.,   42.,   41.,   52., -198.,  200.,  282., -313.,  301., -150.],
    [ 111.,  -78.,   39.,  193., -123., -102., -537., -267.,  711., -422.],
    [-187.,   45.,   33., -333., -136.,  462.,  -33.,  -34., -198.,  202.],
    [-138.,   40.,  253.,  282.,  109., -419.,  -45.,  176., -169.,  248.]
], dtype=np.float)

In [29]:
replacements2 = get_replacements(mask2, f2, g2, reference2, field2)

In [30]:
mat2.subs(replacements2)

Matrix([
[                  12*f21 - 4*f22 - 4*f31 + 516.0],
[                -4*f21 + 12*f31 - 4*f32 - 5038.0],
[        -4*f21 + 14*f22 - 4*f23 - 4*f32 - 2636.0],
[-4*f22 - 4*f31 + 16*f32 - 4*f33 - 4*f42 + 5216.0],
[                -4*f32 + 12*f42 - 4*f43 - 5430.0],
[                -4*f22 + 12*f23 - 4*f33 - 3814.0],
[ -4*f23 - 4*f32 + 16*f33 - 4*f34 - 4*f43 + 312.0],
[ -4*f33 - 4*f42 + 16*f43 - 4*f44 - 4*f53 - 864.0],
[         -4*f43 + 14*f53 - 4*f54 - 4*f63 - 438.0],
[        -4*f53 + 14*f63 - 4*f64 - 4*f73 - 1172.0],
[                -4*f63 + 12*f73 - 4*f74 - 1940.0],
[        -4*f33 + 14*f34 - 4*f35 - 4*f44 - 1922.0],
[-4*f34 - 4*f43 + 16*f44 - 4*f45 - 4*f54 + 3876.0],
[-4*f44 - 4*f53 + 16*f54 - 4*f55 - 4*f64 - 3088.0],
[-4*f54 - 4*f63 + 16*f64 - 4*f65 - 4*f74 - 6744.0],
[-4*f64 - 4*f73 + 16*f74 - 4*f75 - 4*f84 + 5980.0],
[                        -4*f74 + 10*f84 - 5700.0],
[                -4*f34 + 12*f35 - 4*f45 - 5226.0],
[ -4*f35 - 4*f44 + 16*f45 - 4*f46 - 4*f55 + 996.0],
[-4

#### Big ole case

In [29]:
with rio.drivers():
    with rio.open('test2/landsat.tif') as src:
        reference3 = src.read(1)
    with rio.open('test2/20150705_014439_081f_analytic.tif') as src:
        source3 = src.read(1)
        field3 = (pyamg.gallery.poisson(source.shape) * source.ravel()).reshape(source.shape)

In [30]:
replacements3 = get_replacements(mask3, f3, g3, reference3, field3)

In [31]:
mat3.subs(replacements3)

Matrix([
[                          12*f115 - 4*f116 - 4*f125 + 558.0],
[               -4*f115 + 14*f125 - 4*f126 - 4*f135 + 2240.0],
[               -4*f125 + 14*f135 - 4*f136 - 4*f145 - 1608.0],
[               -4*f135 + 14*f145 - 4*f146 - 4*f155 + 2982.0],
[               -4*f145 + 14*f155 - 4*f156 - 4*f165 - 7848.0],
[                  -4*f155 + 14*f165 - 4*f166 - 4*f175 - 2.0],
[               -4*f165 + 14*f175 - 4*f176 - 4*f185 + 2028.0],
[                        -4*f175 + 12*f185 - 4*f186 - 4646.0],
[                          -4*f106 + 12*f96 - 4*f97 - 4268.0],
[                 14*f106 - 4*f107 - 4*f116 - 4*f96 + 7272.0],
[      -4*f106 - 4*f115 + 16*f116 - 4*f117 - 4*f126 - 5292.0],
[      -4*f116 - 4*f125 + 16*f126 - 4*f127 - 4*f136 - 5580.0],
[      -4*f126 - 4*f135 + 16*f136 - 4*f137 - 4*f146 + 2128.0],
[       -4*f136 - 4*f145 + 16*f146 - 4*f147 - 4*f156 + 868.0],
[      -4*f146 - 4*f155 + 16*f156 - 4*f157 - 4*f166 - 1728.0],
[      -4*f156 - 4*f165 + 16*f166 - 4*f167 - 4

In [32]:
sympy.diff(eq3, f3[12][5]).subs(symbolic_replacements3)

-2*c124 - 4*f115 + 14*f125 - 4*f126 - 4*f135 + 4*g115 + 2*g124 - 14*g125 + 4*g126 + 4*g135

In [49]:
display_matrix(mat3.subs(symbolic_replacements3))

In [33]:
mat3s = mat3.subs(replacements3)

In [24]:
mat3s

Matrix([
[                                          -4*f116 + 3480.0],
[                                          -4*f126 + 2786.0],
[                                          -4*f136 + 3458.0],
[                                          -4*f146 + 3446.0],
[                                          -4*f156 - 1592.0],
[                                          -4*f166 + 2488.0],
[                                          -4*f176 + 3922.0],
[                                           -4*f186 + 418.0],
[                                            -4*f97 + 472.0],
[                                 -4*f107 - 4*f116 + 9340.0],
[                        16*f116 - 4*f117 - 4*f126 - 7582.0],
[              -4*f116 + 16*f126 - 4*f127 - 4*f136 - 4774.0],
[              -4*f126 + 16*f136 - 4*f137 - 4*f146 - 1420.0],
[              -4*f136 + 16*f146 - 4*f147 - 4*f156 - 1582.0],
[              -4*f146 + 16*f156 - 4*f157 - 4*f166 - 2808.0],
[               -4*f156 + 16*f166 - 4*f167 - 4*f176 + 702.0],

In [26]:
vec3

[f115,
 f125,
 f135,
 f145,
 f155,
 f165,
 f175,
 f185,
 f96,
 f106,
 f116,
 f126,
 f136,
 f146,
 f156,
 f166,
 f176,
 f186,
 f196,
 f206,
 f87,
 f97,
 f107,
 f117,
 f127,
 f137,
 f147,
 f157,
 f167,
 f177,
 f187,
 f197,
 f207,
 f217,
 f78,
 f88,
 f98,
 f108,
 f118,
 f128,
 f138,
 f148,
 f158,
 f168,
 f178,
 f188,
 f198,
 f208,
 f218,
 f228,
 f69,
 f79,
 f89,
 f99,
 f109,
 f119,
 f129,
 f139,
 f149,
 f159,
 f169,
 f179,
 f189,
 f199,
 f209,
 f219,
 f229,
 f510,
 f610,
 f710,
 f810,
 f910,
 f1010,
 f1110,
 f1210,
 f1310,
 f1410,
 f1510,
 f1610,
 f1710,
 f1810,
 f1910,
 f2010,
 f2110,
 f2210,
 f2310,
 f511,
 f611,
 f711,
 f811,
 f911,
 f1011,
 f1111,
 f1211,
 f1311,
 f1411,
 f1511,
 f1611,
 f1711,
 f1811,
 f1911,
 f2011,
 f2111,
 f2211,
 f2311,
 f512,
 f612,
 f712,
 f812,
 f912,
 f1012,
 f1112,
 f1212,
 f1312,
 f1412,
 f1512,
 f1612,
 f1712,
 f1812,
 f1912,
 f2012,
 f2112,
 f2212,
 f2312,
 f2412,
 f413,
 f513,
 f613,
 f713,
 f813,
 f913,
 f1013,
 f1113,
 f1213,
 f1313,
 f1413,
 f1513,
 f

In [29]:
# x1 = sympy.solve(mat1.subs(replacements1))
# x2 = sympy.solve(mat2.subs(replacements2))
x3 = sympy.solve(mat3s)

In [30]:
x3

[]

In [21]:
def apply_blended_pixels(x, reference, f):
    img = np.copy(reference)
    
    height, width = reference.shape
    
    for j in range(height):
        for i in range(width):
            if x.has_key(f[i][j]):
                img[j][i] = x[f[i][j]]
    
    return img

In [22]:
x3

[]

In [52]:
# img1 = apply_blended_pixels(x1, reference, f1)
# img2 = apply_blended_pixels(x2, reference, f2)
img3 = apply_blended_pixels(x3, reference, f3)

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, sharex=True, sharey=True, figsize=(12, 4))
ax1.imshow(img1, vmin=300, vmax=750)
ax2.imshow(img2, vmin=300, vmax=750)

In [ ]:
print img1

In [ ]:
print img2